<a href="https://colab.research.google.com/github/filipchudzynski/stock-market-non-gaussianity-analyzer_v2/blob/main/basic_analysis_of_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get the data from www.histdata.com

In [ ]:
! pip install selenium pandas openpyxl webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [ ]:
!apt-get update
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb


Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,213 B]
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,038 B in 1s (2,502 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
R

In [ ]:
# run in terminal !CHROME_VERSION=$(google-chrome --version | grep -oP '\d+\.\d+\.\d+')


!wget -q "https://storage.googleapis.com/chrome-for-testing-public/144.0.7559.132/linux64/chromedriver-linux64.zip" -O chromedriver.zip
!unzip chromedriver.zip
!mv chromedriver-linux64/chromedriver /usr/local/bin/
!chmod +x /usr/local/bin/chromedriver


Archive:  chromedriver.zip
  inflating: chromedriver-linux64/LICENSE.chromedriver  
  inflating: chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: chromedriver-linux64/chromedriver  


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from pathlib import Path

def get_driver(download_dir="/content/downloads"):
    Path(download_dir).mkdir(exist_ok=True)

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)

    return webdriver.Chrome(options=options)


In [ ]:
def download_year(year):
    print(f"\n=== Processing {year} ===")
    driver.get(BASE_URL.format(year=year))

    time.sleep(2)

    # Debug: show what page we actually loaded
    print("Current URL:", driver.current_url)
    print("Page title:", driver.title)

    # Extract the visible year from the page
    try:
        year_text = driver.find_element(By.XPATH, "//p[b[contains(text(),'Year/Month')]]").text
        print("Page shows:", year_text)
    except:
        print("Could not read Year/Month from page")

    # Extract the download filename text
    try:
        file_label = driver.find_element(By.ID, "a_file").text
        print("Download link text:", file_label)
    except:
        print("Could not read download link text")

    # Now proceed with clicking
    try:
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "a_file"))
        )
    except Exception as e:
        print(f"Could not find download link for {year}: {e}")
        return None

    link.click()
    print("Clicked download link")

    # Wait for ZIP to appear
    zip_path = Path(DOWNLOAD_DIR) / f"{year}.zip"
    for _ in range(40):
        for f in Path(DOWNLOAD_DIR).glob("*.zip"):
            f.rename(zip_path)
            print("Downloaded ZIP →", zip_path)
            return zip_path
        time.sleep(1)

    print("Failed to download", year)
    return None


In [ ]:
import time
import zipfile
from pathlib import Path
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

DOWNLOAD_DIR = "/content/downloads"
BASE_URL = "https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/spxusd/{year}"

driver = get_driver(DOWNLOAD_DIR)  # reuse the headless driver you already defined

def download_year(year):
    print(f"Processing {year}...")
    driver.get(BASE_URL.format(year=year))


    time.sleep(2)

    # Debug: show what page we actually loaded
    print("Current URL:", driver.current_url)
    print("Page title:", driver.title)


    # 1. Close cookie banner if present
    try:
        accept_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "cookie_action_close_header"))
        )
        accept_btn.click()
        time.sleep(1)
        print("Cookie banner dismissed")
    except:
        print("No cookie banner found")

    # Extract the visible year from the page
    try:
        year_text = driver.find_element(By.XPATH, "//p[b[contains(text(),'Year/Month')]]").text
        print("Page shows:", year_text)
    except:
        print("Could not read Year/Month from page")

    # Extract the download filename text
    try:
        file_label = driver.find_element(By.ID, "a_file").text
        print("Download link text:", file_label)
    except:
        print("Could not read download link text")

    # Now proceed with clicking
    try:
        link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "a_file"))
        )
    except Exception as e:
        print(f"Could not find download link for {year}: {e}")
        return None

    # 3. Click the download link
    link.click()
    print("Clicked download link")

    # 4. Wait for ZIP to appear
    zip_path = Path(DOWNLOAD_DIR) / f"{year}.zip"
    for _ in range(40):
        for f in Path(DOWNLOAD_DIR).glob(f"*{year}.zip"):
            print(f"{f} found")
            f.rename(zip_path)
            print("Downloaded ZIP →", zip_path)
            return zip_path
        time.sleep(1)

    print("Failed to download", year)
    return None


def extract_and_convert(zip_path):
    year = zip_path.stem
    with zipfile.ZipFile(zip_path, "r") as z:
        xlsx_name = next(n for n in z.namelist() if n.lower().endswith(".xlsx"))
        xlsx_path = Path(DOWNLOAD_DIR) / f"{year}.xlsx"
        z.extract(xlsx_name, DOWNLOAD_DIR)
        (Path(DOWNLOAD_DIR) / xlsx_name).rename(xlsx_path)
        print("Extracted XLSX →", xlsx_path)

    df = pd.read_excel(xlsx_path)
    display(df.head())
    csv_path = Path(DOWNLOAD_DIR) / f"{year}.csv"
    df.to_csv(csv_path, index=False)
    print("Converted to CSV →", csv_path)

# run it
for year in range(2020, 2025):
    zip_file = download_year(year)
    if zip_file:
        extract_and_convert(zip_file)

driver.quit()


Processing 2020...
Current URL: https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/spxusd/2020
Page title: Download Free Forex Historical Data – HistData.com
Cookie banner dismissed
Page shows: Year/Month: 2020
 
Download link text: HISTDATA_COM_XLSX_SPXUSD_M1_2020.zip
Clicked download link
/content/downloads/2020.zip found
Downloaded ZIP → /content/downloads/2020.zip
Extracted XLSX → /content/downloads/2020.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,2020-01-01 18:00:00,3234.552,3235.537,3234.534,3234.837,0
0,2020-01-01 18:01:00,3234.837,3234.837,3233.337,3233.337,0
1,2020-01-01 18:02:00,3233.337,3233.852,3233.334,3233.740,0
2,2020-01-01 18:03:00,3233.740,3234.600,3233.552,3234.346,0
3,2020-01-01 18:04:00,3234.346,3234.352,3233.852,3233.855,0
4,2020-01-01 18:05:00,3233.855,3234.100,3233.837,3233.900,0


Converted to CSV → /content/downloads/2020.csv
Processing 2021...
Current URL: https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/spxusd/2021
Page title: Download Free Forex Historical Data – HistData.com
No cookie banner found
Page shows: Year/Month: 2021
 
Download link text: HISTDATA_COM_XLSX_SPXUSD_M1_2021.zip
Clicked download link
/content/downloads/HISTDATA_COM_XLSX_SPXUSD_M12021.zip found
Downloaded ZIP → /content/downloads/2021.zip
Extracted XLSX → /content/downloads/2021.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


...

,2022-01-02 18:00:00,4779.636,4785.539,4779.636.1,4781.251,0
0,2022-01-02 18:01:00,4781.136,4784.099,4780.133,4782.736,0
1,2022-01-02 18:02:00,4782.836,4784.242,4781.342,4781.348,0
2,2022-01-02 18:03:00,4781.648,4782.151,4780.142,4780.242,0
3,2022-01-02 18:04:00,4780.145,4781.699,4779.836,4781.633,0
4,2022-01-02 18:05:00,4781.233,4782.699,4781.133,4781.854,0


Converted to CSV → /content/downloads/2022.csv
Processing 2023...
Current URL: https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/spxusd/2023
Page title: Download Free Forex Historical Data – HistData.com
No cookie banner found
Page shows: Year/Month: 2023
 
Download link text: HISTDATA_COM_XLSX_SPXUSD_M1_2023.zip
Clicked download link
/content/downloads/HISTDATA_COM_XLSX_SPXUSD_M12023.zip found
Downloaded ZIP → /content/downloads/2023.zip
Extracted XLSX → /content/downloads/2023.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,2023-01-02 18:00:00,3872.998,3877.176,3863.86,3865.983,0
0,2023-01-02 18:01:00,3866.128,3867.372,3865.378,3865.980,0
1,2023-01-02 18:02:00,3865.878,3866.878,3865.360,3865.881,0
2,2023-01-02 18:03:00,3865.742,3865.742,3862.860,3863.613,0
3,2023-01-02 18:04:00,3863.363,3863.363,3860.742,3860.878,0
4,2023-01-02 18:05:00,3861.113,3864.378,3860.875,3864.119,0


Converted to CSV → /content/downloads/2023.csv
Processing 2024...
Current URL: https://www.histdata.com/download-free-forex-historical-data/?/excel/1-minute-bar-quotes/spxusd/2024
Page title: Download Free Forex Historical Data – HistData.com
No cookie banner found
Page shows: Year/Month: 2024
 
Download link text: HISTDATA_COM_XLSX_SPXUSD_M1_2024.zip
Clicked download link
Failed to download 2024


# data quality

In [ ]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("/content/downloads")

csv_files = sorted(DATA_DIR.glob("*.csv"))
dfs = []
print(csv_files)
for f in csv_files:
    df = pd.read_csv(f, header=None)
    df.columns = [ "timestamp", "open", "high", "low", "close","vol" ]
    dfs.append(df)

# data = pd.concat(dfs, ignore_index=True)
# print("Loaded rows:", len(data))
for df in dfs:
  display(df.head())


[PosixPath('/content/downloads/2020.csv'), PosixPath('/content/downloads/2021.csv'), PosixPath('/content/downloads/2022.csv'), PosixPath('/content/downloads/2023.csv')]


/tmp/ipython-input-3887703487.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, header=None)


,timestamp,open,high,low,close,vol
0,2020-01-01 18:00:00,3234.552,3235.537,3234.534,3234.837,0
1,2020-01-01 18:01:00,3234.837,3234.837,3233.337,3233.337,0
2,2020-01-01 18:02:00,3233.337,3233.852,3233.334,3233.740,0
3,2020-01-01 18:03:00,3233.740,3234.600,3233.552,3234.346,0
4,2020-01-01 18:04:00,3234.346,3234.352,3233.852,3233.855,0


,timestamp,open,high,low,close,vol
0,2021-01-03 18:00:00,3758.942,3762.397,3756.854,3757.954,0
1,2021-01-03 18:01:00,3758.137,3759.436,3756.231,3756.254,0
2,2021-01-03 18:02:00,3756.231,3756.231,3749.936,3752.242,0
3,2021-01-03 18:03:00,3752.446,3754.754,3751.139,3752.636,0
4,2021-01-03 18:04:00,3752.631,3754.297,3752.434,3753.433,0


,timestamp,open,high,low,close,vol
0,2022-01-02 18:00:00,4779.636,4785.539,4779.636.1,4781.251,0
1,2022-01-02 18:01:00,4781.136,4784.099,4780.133,4782.736,0
2,2022-01-02 18:02:00,4782.836,4784.242,4781.342,4781.348,0
3,2022-01-02 18:03:00,4781.648,4782.151,4780.142,4780.242,0
4,2022-01-02 18:04:00,4780.145,4781.699,4779.836,4781.633,0


,timestamp,open,high,low,close,vol
0,2023-01-02 18:00:00,3872.998,3877.176,3863.860,3865.983,0
1,2023-01-02 18:01:00,3866.128,3867.372,3865.378,3865.980,0
2,2023-01-02 18:02:00,3865.878,3866.878,3865.360,3865.881,0
3,2023-01-02 18:03:00,3865.742,3865.742,3862.860,3863.613,0
4,2023-01-02 18:04:00,3863.363,3863.363,3860.742,3860.878,0


In [ ]:
for data in dfs:
  data["timestamp"] = pd.to_datetime(data["timestamp"])
  data = data.sort_values("timestamp")
  expected_delta = pd.Timedelta(minutes=1)
  actual_delta = data["timestamp"].diff()

  gaps = data[actual_delta > expected_delta]
  display(gaps.head())
  gap_stats = actual_delta[actual_delta > expected_delta].describe()
  print(gap_stats)


,timestamp,open,high,low,close,vol
17,2020-01-01 18:18:00,3237.337,3238.034,3237.337,3238.034,0
54,2020-01-01 18:56:00,3236.355,3236.543,3236.349,3236.355,0
56,2020-01-01 19:00:00,3236.034,3236.852,3236.034,3236.046,0
63,2020-01-01 19:08:00,3235.846,3235.846,3235.600,3235.600,0
73,2020-01-01 19:19:00,3235.855,3236.034,3235.855,3236.034,0


count                         5283
mean     0 days 00:37:12.538330494
std      0 days 05:03:41.306658171
min                0 days 00:02:00
25%                0 days 00:02:00
50%                0 days 00:02:00
75%                0 days 00:02:00
max                3 days 04:46:00
Name: timestamp, dtype: object


,timestamp,open,high,low,close,vol
11,2021-01-03 18:12:00,3756.731,3757.948,3756.731,3757.442,0
229,2021-01-03 21:51:00,3754.431,3754.742,3753.931,3754.248,0
256,2021-01-03 22:19:00,3756.499,3756.499,3755.940,3755.948,0
285,2021-01-03 22:49:00,3756.746,3756.754,3756.433,3756.454,0
293,2021-01-03 22:58:00,3756.448,3756.742,3756.446,3756.742,0


count                         7455
mean     0 days 00:26:10.229376257
std      0 days 04:10:43.657652507
min                0 days 00:02:00
25%                0 days 00:02:00
50%                0 days 00:02:00
75%                0 days 00:02:00
max                3 days 01:47:00
Name: timestamp, dtype: object


,timestamp,open,high,low,close,vol
326,2022-01-02 23:27:00,4785.654,4785.654,4785.639,4785.639,0
331,2022-01-02 23:33:00,4786.033,4786.145,4786.033,4786.039,0
336,2022-01-02 23:40:00,4786.351,4786.351,4786.048,4786.136,0
338,2022-01-02 23:43:00,4786.242,4786.354,4786.142,4786.154,0
372,2022-01-03 00:18:00,4786.148,4786.648,4786.136,4786.345,0


count                         1234
mean     0 days 02:26:30.729335494
std      0 days 10:06:34.563415199
min                0 days 00:02:00
25%                0 days 00:02:00
50%                0 days 00:02:00
75%                0 days 00:03:00
max                3 days 01:47:00
Name: timestamp, dtype: object


,timestamp,open,high,low,close,vol
1335,2023-01-03 18:00:00,3820.439,3820.439,3817.276,3818.200,0
1374,2023-01-03 18:40:00,3819.583,3819.715,3819.222,3819.453,0
1483,2023-01-03 20:30:00,3827.598,3827.721,3827.460,3827.721,0
1515,2023-01-03 21:03:00,3830.969,3830.969,3830.235,3830.238,0
1584,2023-01-03 22:13:00,3828.029,3828.086,3826.913,3827.283,0


count                         7003
mean     0 days 00:33:36.472940168
std      0 days 04:23:31.316198225
min                0 days 00:02:00
25%                0 days 00:02:00
50%                0 days 00:02:00
75%                0 days 00:03:00
max                3 days 01:47:00
Name: timestamp, dtype: object
